In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))
from utils.loading import load_net
from utils.train_val import validate_epoch
from utils.data import make_generators_DF_MNIST

import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torch.nn import functional as F
import torch.optim as optim

import foolbox
import json
from PIL import Image

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.current_device())

BASE_PATH = Path('/media/rene/data/adv_gen/MNIST/mnist_normal/feature_models')

cuda:1
0


In [4]:
files_df_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/files_df.pkl'
batch_size = 64
num_workers = 2
IM_SIZE = 28

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)

dataloaders = make_generators_DF_MNIST(files_df, batch_size, num_workers, size=IM_SIZE,
                                        path_colname='path', adv_path_colname=None, return_loc=False, normalize=True)

In [8]:
model_name_list = ['SimpleNetMNIST-10_model_best.pth.tar',
                   'SimpleNetMNIST-12_model_best.pth.tar',
                   'SimpleNetMNIST-16_model_best.pth.tar',
                   'SimpleNetMNIST-20_model_best.pth.tar',
                   'TopkNetMNIST-16-10_model_best.pth.tar',
                   'TopkNetMNIST-16-8_model_best.pth.tar',
                   'TopkNetMNIST-12-6_model_best.pth.tar',
                   'TopkNetMNIST-10-5_model_best.pth.tar'
                  ]

for model_name in model_name_list:
    model_loc = BASE_PATH / model_name
    model = load_net(model_loc).to(device).eval()
    acc, loss = validate_epoch(dataloaders['val'], model, device)
    print(f'Model Name: {model_name}: Accuracy: {acc}')

VALID:  * TOP1 98.690 TOP5 100.000 Loss (0.0000)	 Time (0.010)	
Model Name: SimpleNetMNIST-10_model_best.pth.tar: Accuracy: 98.68999481201172
VALID:  * TOP1 98.900 TOP5 100.000 Loss (0.0000)	 Time (0.010)	
Model Name: SimpleNetMNIST-12_model_best.pth.tar: Accuracy: 98.89999389648438
VALID:  * TOP1 98.870 TOP5 100.000 Loss (0.0000)	 Time (0.009)	
Model Name: SimpleNetMNIST-16_model_best.pth.tar: Accuracy: 98.8699951171875
VALID:  * TOP1 98.850 TOP5 100.000 Loss (0.0000)	 Time (0.010)	
Model Name: SimpleNetMNIST-20_model_best.pth.tar: Accuracy: 98.8499984741211
VALID:  * TOP1 98.860 TOP5 100.000 Loss (0.0000)	 Time (0.010)	
Model Name: TopkNetMNIST-16-10_model_best.pth.tar: Accuracy: 98.86000061035156
VALID:  * TOP1 98.830 TOP5 100.000 Loss (0.0000)	 Time (0.009)	
Model Name: TopkNetMNIST-16-8_model_best.pth.tar: Accuracy: 98.82999420166016
VALID:  * TOP1 98.800 TOP5 99.990 Loss (0.0000)	 Time (0.010)	
Model Name: TopkNetMNIST-12-6_model_best.pth.tar: Accuracy: 98.79999542236328
VALID:  

In [5]:
model_name_list = ['SimpleNetMNIST-10_model_best.pth.tar',
                   'SimpleNetMNIST-12_model_best.pth.tar',
                   'SimpleNetMNIST-16_model_best.pth.tar',
                   'SimpleNetMNIST-20_model_best.pth.tar',
                   'TopkNetMNIST-30-8_model_best.pth.tar',
                   'TopkNetMNIST-16-10_model_best.pth.tar',
                   'TopkNetMNIST-16-8_model_best.pth.tar',
                   'TopkNetMNIST-12-6_model_best.pth.tar',
                   'TopkNetMNIST-10-5_model_best.pth.tar',
                  ]

for model_name in model_name_list:
    model_loc = BASE_PATH / model_name
    model = load_net(model_loc).to(device).eval()
    acc, loss = validate_epoch(dataloaders['val'], model, device)
    print(f'Model Name: {model_name}: Accuracy: {acc}')

VALID:  * TOP1 98.670 TOP5 100.000 Loss (0.0000)	 Time (0.010)	
Model Name: SimpleNetMNIST-10_model_best.pth.tar: Accuracy: 98.66999816894531
VALID:  * TOP1 98.970 TOP5 99.980 Loss (0.0000)	 Time (0.010)	
Model Name: SimpleNetMNIST-12_model_best.pth.tar: Accuracy: 98.97000122070312
VALID:  * TOP1 98.770 TOP5 99.990 Loss (0.0000)	 Time (0.011)	
Model Name: SimpleNetMNIST-16_model_best.pth.tar: Accuracy: 98.7699966430664
VALID:  * TOP1 98.930 TOP5 99.990 Loss (0.0000)	 Time (0.010)	
Model Name: SimpleNetMNIST-20_model_best.pth.tar: Accuracy: 98.93000030517578
VALID:  * TOP1 98.830 TOP5 100.000 Loss (0.0000)	 Time (0.010)	
Model Name: TopkNetMNIST-30-8_model_best.pth.tar: Accuracy: 98.82999420166016
VALID:  * TOP1 98.900 TOP5 99.990 Loss (0.0000)	 Time (0.010)	
Model Name: TopkNetMNIST-16-10_model_best.pth.tar: Accuracy: 98.89999389648438
VALID:  * TOP1 98.740 TOP5 100.000 Loss (0.0000)	 Time (0.010)	
Model Name: TopkNetMNIST-16-8_model_best.pth.tar: Accuracy: 98.73999786376953
VALID:  * 

In [ ]:
model_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/feature_models'